In [2]:
#Which types of vehicles have the highest number of accidents and worst severity?

#Is there a particular day of the week and time with a higher number of accidents? (Monday after 5 vs Friday after 5)

#Does higher amount of horsepower correlate to higher risk of accidents?

#How does age and sex affect risk of accident and severity? (could bin ages to inexperienced, moderately experienced, experienced, senior citizen)

#Do bigger vehicles correlate to higher rate of casualties/severity? (Are motorcycles/mopeds more dangerous?)

#Does purpose of trip affectt risk of accident?

#Do lighting conditions (time of day) have an effect on accidents and severity? (daylight vs darkness)

#Which type of vehicles maneuvers cause the most accidents? (see vehicle maneuver context csv)

#How do different types of intersections effect risk of accident and severity?

#Do older vehicles have any type of effect on accidents?

# 

### top 5 locations for accidents - heatmaps 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import date
from collections import defaultdict
import seaborn as sns

#main data source
files = glob.glob('source/*.csv')
#['source/Vehicles0515.csv',
# 'source/Accidents0515.csv',
# 'source/Casualties0515.csv']

context_files = glob.glob('source/context/*.csv')
#context_files

#!pwd

In [3]:
accident_severity = context_files[18]
accident_severity = pd.read_csv(accident_severity)
accident_severity = accident_severity.set_index('code')
accident_severity
type(accident_severity)
accident_severity = accident_severity.rename(columns={'label': 'Category'})
accident_severity

,Category
code,
1,Fatal
2,Serious
3,Slight


In [4]:
# accident Data set
accident_set = pd.read_csv(files[1])
#print(accident_set.columns)

In [5]:
# 1st clean
accident_set = accident_set.dropna(axis=0, how='all')
accident_set.head(3)
accident_set = accident_set.dropna()
accident_set.columns

Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location'],
      dtype='object')

In [6]:
# Vehicle Data set
vehicle_set = pd.read_csv(files[0])

In [9]:
# 1st clean
vehicle_set = vehicle_set.drop_duplicates()
vehicle_set = vehicle_set.dropna(axis=0, how='all')
vehicle_set = vehicle_set.dropna(axis=1, how='any')


In [10]:
# show first 3 rows
vehicle_set.head(14)

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type
0,200501BS00001,1,9,0,18,0,0,0,0,0,...,1,15,2,74,10,-1,-1,-1,7,1
1,200501BS00002,1,11,0,4,0,3,0,0,0,...,1,1,1,42,7,8268,2,3,-1,-1
2,200501BS00003,1,11,0,17,0,0,0,4,0,...,1,1,1,35,6,8300,2,5,2,1
3,200501BS00003,2,9,0,2,0,0,0,0,0,...,1,15,1,62,9,1762,1,6,1,1
4,200501BS00004,1,9,0,18,0,0,0,0,0,...,1,15,2,49,8,1769,1,4,2,1
5,200501BS00005,1,3,0,18,0,0,1,10,0,...,1,15,1,49,8,85,1,10,-1,-1
6,200501BS00006,1,9,0,5,0,0,0,0,0,...,1,15,1,51,8,2976,1,1,4,1
7,200501BS00006,2,3,0,18,0,0,0,0,0,...,1,15,2,30,6,124,1,2,1,1
8,200501BS00007,1,3,0,18,0,1,0,4,0,...,1,15,1,31,6,-1,-1,-1,-1,-1
9,200501BS00007,2,9,0,2,0,1,0,0,0,...,1,15,1,41,7,4266,1,4,6,1


In [20]:
# Join accident and vehicle on Accident
merge_set = pd.merge(vehicle_set,
                    accident_set,
                    on='Accident_Index', how= 'outer')

In [21]:
casualties = pd.read_csv(files[2])
casualties = casualties.dropna(axis=0, how='all')
casualties = casualties.drop(columns=['extra'])
#casualties = casualties.set_index('Vehicle_Reference')
casualties = casualties.drop_duplicates()
casualties.head(3)

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type
0,200501BS00001,1,1,3,1,37,7,2,1,1,0,0,-1,0,1
1,200501BS00002,1,1,2,1,37,7,3,0,0,0,4,-1,11,1
2,200501BS00003,2,1,1,1,62,9,3,0,0,0,0,-1,9,1


In [22]:
merge_set = pd.merge(merge_set, casualties, 
                     on='Accident_Index', suffixes=["_L", "_R"])

In [24]:
# show first 10
merge_set.head(10)

,Accident_Index,Vehicle_Reference_L,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type
0,200501BS00001,1,9,0,18,0,0,0,0,0,...,37,7,2,1,1,0,0,-1,0,1
1,200501BS00002,1,11,0,4,0,3,0,0,0,...,37,7,3,0,0,0,4,-1,11,1
2,200501BS00003,1,11,0,17,0,0,0,4,0,...,62,9,3,0,0,0,0,-1,9,1
3,200501BS00003,2,9,0,2,0,0,0,0,0,...,62,9,3,0,0,0,0,-1,9,1
4,200501BS00004,1,9,0,18,0,0,0,0,0,...,30,6,3,5,2,0,0,-1,0,1
5,200501BS00005,1,3,0,18,0,0,1,10,0,...,49,8,3,0,0,0,0,-1,3,-1
6,200501BS00006,1,9,0,5,0,0,0,0,0,...,30,6,3,0,0,0,0,-1,3,1
7,200501BS00006,2,3,0,18,0,0,0,0,0,...,30,6,3,0,0,0,0,-1,3,1
8,200501BS00007,1,3,0,18,0,1,0,4,0,...,31,6,3,0,0,0,0,-1,3,-1
9,200501BS00007,2,9,0,2,0,1,0,0,0,...,31,6,3,0,0,0,0,-1,3,-1


In [25]:
#outer merging keeps common values in both left and right. Only rows with city should be return.  
print(merge_set.shape)
# (3262270, 53)
print(vehicle_set.shape)
# (3262270, 22)
print(accident_set.shape)
#(1780653, 32)
# validate by checking the number of values in commomn 
merge_set.count()
#combined_dat
vehicle_set['Accident_Index'].isin(accident_set['Accident_Index']).value_counts()

(4650859, 67)
(3262270, 22)
(1651142, 32)


True     3041941
False     220329
Name: Accident_Index, dtype: int64

In [26]:
# assign vehicle types context 
vehicle_type = pd.read_csv(context_files[2])

In [27]:
# index will now be know as the code type that describes each vehicle
vehicle_type = vehicle_type.set_index('code')

In [28]:
vehicle_type

,label
code,
1,Pedal cycle
2,Motorcycle 50cc and under
3,Motorcycle 125cc and under
4,Motorcycle over 125cc and up to 500cc
5,Motorcycle over 500cc
8,Taxi/Private hire car
9,Car
10,Minibus (8 - 16 passenger seats)
11,Bus or coach (17 or more pass seats)


In [29]:
# Created another set for our1st Question. Inner join on the type of vehicles
types_of_vehicles = merge_set.join(vehicle_type, 
                                   on='Vehicle_Type', how='inner')

In [30]:
types_of_vehicles = types_of_vehicles[types_of_vehicles.label != 'Data missing or out of range']

In [31]:
# first couple of rows
types_of_vehicles.head(5)  

,Accident_Index,Vehicle_Reference_L,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,label
0,200501BS00001,1,9,0,18,0,0,0,0,0,...,7,2,1,1,0,0,-1,0,1,Car
3,200501BS00003,2,9,0,2,0,0,0,0,0,...,9,3,0,0,0,0,-1,9,1,Car
4,200501BS00004,1,9,0,18,0,0,0,0,0,...,6,3,5,2,0,0,-1,0,1,Car
6,200501BS00006,1,9,0,5,0,0,0,0,0,...,6,3,0,0,0,0,-1,3,1,Car
9,200501BS00007,2,9,0,2,0,1,0,0,0,...,6,3,0,0,0,0,-1,3,-1,Car


In [32]:
## last rows
types_of_vehicles.tail()

,Accident_Index,Vehicle_Reference_L,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,label
4476599,2015320011119,2,23,0,18,0,0,0,0,0,...,9,3,0,0,0,0,0,23,1,Electric motorcycle
4481908,2015331500216,2,23,0,9,0,3,0,0,0,...,4,3,0,0,0,0,0,23,-1,Electric motorcycle
4538883,201543W048065,1,23,0,18,0,0,0,0,0,...,5,2,0,0,0,0,0,23,1,Electric motorcycle
4552110,2015450022238,1,23,0,18,9,0,5,0,0,...,11,2,0,0,0,0,0,23,1,Electric motorcycle
4616459,201560S036521,1,23,0,5,0,1,0,0,0,...,7,2,0,0,0,0,0,23,-1,Electric motorcycle


In [ ]:
# Checking - 6 random sample of rows
vehicle_sample = types_of_vehicles.sample(100)

In [ ]:
#num or rows/columns in a tuple # memory usage: 1.3+ GB
types_of_vehicles.shape

In [ ]:
# calculate measures of central tendency
#types_of_vehicles.describe()   

In [ ]:
# data types and memory consumption 
types_of_vehicles.info()

In [ ]:
most_acc= types_of_vehicles.groupby(['label','Vehicle_Type'])['Accident_Severity'].value_counts()

In [ ]:
mean_acc= types_of_vehicles.groupby('label')['Accident_Severity'].sum()

In [ ]:
most_vehicle_accident = types_of_vehicles.groupby('label')['Accident_Severity'].value_counts()

In [ ]:
most_accidents = types_of_vehicles.groupby('Accident_Severity')['label'].value_counts()

In [ ]:
types_of_vehicles['label'].value_counts()

In [ ]:
types_of_vehicles = types_of_vehicles.dropna()

In [ ]:
types_of_vehicles = types_of_vehicles.join(accident_severity, 
                                           on='Accident_Severity', 
                                           how='inner')


In [ ]:
types_of_vehicles


In [ ]:
#number of accident and most severe per vehicle
most_vehicle_accident

In [ ]:
mostaccidents = types_of_vehicles.groupby(
    ['Category','label'])['Vehicle_Reference'].count()

#mostaccidents = pd.merge(mostaccidents, accident_severity, on='label', how='outer')

#accidents_plt = mostaccidents.unstack('Accident_Severity').loc[:, 3]
accidents_plt = mostaccidents.unstack('Category')
accidents_plt.plot(kind='barh', stacked=True,colormap='Paired',figsize=(30,60), fontsize=30)

In [ ]:
mostaccidents.Fatal
#mostaccidents

In [ ]:
# Most Accidents categorize on severety with 3 as worst 
#most_acc = pd.DataFrame(most_accidents)
#most_acc

In [ ]:
casualties.head(3)
casualties = casualties.dropna(axis=0, how='all')
#casualties = casualties.drop(columns=['extra'])
casualties.head(3)

In [ ]:
print(type(types_of_vehicles.Day_of_Week))

In [ ]:
# day of week file to join
days_of_week = pd.read_csv(context_files[15])

In [ ]:
days_of_week = days_of_week.set_index('code')

In [ ]:
type(days_of_week)

In [ ]:
days_of_week = days_of_week.rename(columns={'label': 'Days'})

In [ ]:
part_of_days = types_of_vehicles.join(
    days_of_week, on='Day_of_Week', how='inner')

In [ ]:
part_of_days.head(4)

In [ ]:
# Created another set for our1st Question. Inner join on the type of vehicles
#set = part_of_days.join(casualties, on='Vehicle_Reference', how='inner')

In [ ]:
#Data set main for now
part_days = part_of_days.loc[:,['Days','label','Number_of_Vehicles' ,'Accident_Severity','Number_of_Casualties','Date','Time']]

In [ ]:
part_days = part_days.rename(columns={'label':'Vehicle_Type'})

In [ ]:
accidents_perday = part_of_days.groupby('Days')['Category'].value_counts()
type(accidents_perday)
apd = accidents_perday.unstack('Days')
apd

In [ ]:
apd.mean().plot(kind='bar')

In [ ]:
#Is there a particular day of the week and time with a higher number of accidents? (Monday after 5 vs Friday after 5)
#part_of_days = part_of_days.groupby('Days')['Time'].value_counts()

In [ ]:
type(part_of_days)

In [ ]:
#part_of_days

In [ ]:
type(part_days)

In [ ]:
part_days.set_index('Date').head(2)

In [ ]:
#dayOf = part_days.groupby(['Date','Days','Time'])['Accident_Severity'].sum()
dayOf = part_days.groupby(['Date','Days'])['Accident_Severity'].count()
dayOf

In [ ]:
#dayOf_plt = dayOf.unstack('Accident_Severity')
d =dayOf.unstack('Days')

d.head(3)
#dayOf_plt.plot()
#d.fillna(d.mean()).plot(kind='line')
doi = dayOf.index
dov = dayOf.values

d.cumsum()
#plt.figure(); d.plot()
#d.plot.bar(stacked=True);

In [ ]:
#Does higher amount of horsepower correlate to higher risk of accidents?

risk_of_accidents = types_of_vehicles

In [ ]:
risk_of_accidents.head(2)

In [ ]:
# Drop all rows on where the vehicle type does not refrences a value that is equivalent a moving vehicle 
risk_of_accidents = risk_of_accidents[risk_of_accidents.Vehicle_Type != -1]

In [ ]:
risk_of_accidents = risk_of_accidents.loc[:,['label','Engine_Capacity_(CC)','Accident_Severity']]

In [ ]:
risk_of_accidents 

In [ ]:
risk_of_accidents = risk_of_accidents.rename(columns={'label':'Vehicle_Type', 'Engine_Capacity_(CC)':'Engine_Capacity'})

In [ ]:
risk_of_accidents.info()

In [ ]:
risk_of_accidents = risk_of_accidents[risk_of_accidents.Engine_Capacity != -1]

In [ ]:
risk_of_accidents = risk_of_accidents.set_index('Vehicle_Type')

In [ ]:
risk_of_accidents['Accident_Severity'].value_counts()


In [ ]:
#risk_of_accidents = risk_of_accidents.set_index('Vehicle_Type')

In [ ]:
high_risk_of_accidents = risk_of_accidents[risk_of_accidents['Accident_Severity']==3]


In [ ]:
# Drop all rows that the index is not equal to a moving vehicle 
#risk_of_accidents = risk_of_accidents.drop(
#    ['Pedal cycle','Ridden horse',
#     'Mobility scooter','Data missing or out of range'], axis=0)


In [ ]:
high_risk_of_accidents.head(3)

In [ ]:
risk_of_accidents['Accident_Severity'
                      ].corr(risk_of_accidents
                             ['Engine_Capacity'])

In [ ]:
risk_of_accidents.head(3)

In [ ]:
risk_of_accidents

In [ ]:
#risk_of_accidents['Accident_Severity'].corr(risk_of_accidents['Engine_Capacity'])

In [ ]:
#risk_of_accidents.count()

In [ ]:
#How does age and sex affect risk of accident and severity? (could bin ages to inexperienced, moderately experienced, experienced, senior citizen)

#Do bigger vehicles correlate to higher rate of casualties/severity? (Are motorcycles/mopeds more dangerous?)

#Does purpose of trip affectt risk of accident?



